# Data Wrangling with Pandas - analyze the data

In [1]:
import pandas as pd

In [2]:
cars = pd.read_pickle('cars.pkl')

In [3]:
cars.head()

,aspiration,carbody,enginesize,curbweight,price
0,std,convertible,130,2548,13495.0
1,std,convertible,130,2548,16500.0
2,std,hatchback,152,2823,16500.0
3,std,sedan,109,2337,13950.0
4,std,sedan,136,2824,17450.0


### Pandas melt() method: Melt columns to create long data

* **melt(params)**  -> melts the data in two or more columns into two columns.

Parameters:

* **id_var** -> The column or columns that won’t be melted.
* **value_vars** -> The columns to melt. If none are specified, all will be melted.
* **var_name** -> The name of the column that will contain the melted column names, or “variable” by default.
* **value_name** -> The name of the column that will contain the melted column values, or “value” by default.

In [4]:
cars_melted = pd.melt(cars, id_vars='price',
                      value_vars=['enginesize','curbweight'],
                      var_name='feature', value_name='featureValue')
cars_melted

,price,feature,featureValue
0,13495.0,enginesize,130
1,16500.0,enginesize,130
2,16500.0,enginesize,152
3,13950.0,enginesize,109
4,17450.0,enginesize,136
...,...,...,...
405,16845.0,curbweight,2952
406,19045.0,curbweight,3049
407,21485.0,curbweight,3012
408,22470.0,curbweight,3217


### Pandas groupby() method

* **groupby(params)**  -> Returns a GroupBy object that supports aggregate methods such as sum().

Parameters:

* **by** -> The column or list of columns to group by.
* **as_index** -> If False, doesn’t create an index based on the groupby columns. If True (the default), it does.

In [5]:
# get the data
fires = pd.read_pickle('fires_prepared.pkl')
fires = fires.reset_index(drop=True)

In [6]:
fires.head(3)

,fire_name,fire_year,state,discovery_date,contain_date,acres_burned,fire_month,days_burning
0,Power,2004,CA,2004-10-06,2004-10-21,16823.0,10,15.0
1,Freds,2004,CA,2004-10-13,2004-10-17,7700.0,10,4.0
2,Bachelor,2004,NM,2004-07-20,2004-07-20,10.0,7,0.0


In [7]:
fires.groupby('state').mean().head(3)

,fire_year,acres_burned,fire_month,days_burning
state,,,,
AK,2004.742504,11367.199362,6.264198,32.081535
AL,2003.885422,42.348169,5.022529,0.272676
AR,2005.850793,50.281673,5.581081,0.400992


In [8]:
fires.groupby(['state','fire_year','fire_month']).max().head(3)

C:\Users\Vijay Koju\AppData\Local\Temp\ipykernel_20340\2829993106.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  fires.groupby(['state','fire_year','fire_month']).max().head(3)


discovery_date contain_date  acres_burned  \
state fire_year fire_month                                             
AK    1992      5              1992-05-31   1992-06-22        1410.0   
                6              1992-06-29   1992-09-17       48087.0   
                7              1992-07-30   1992-09-17       35090.0   

                            days_burning  
state fire_year fire_month                
AK    1992      5                   50.0  
                6                   82.0  
                7                   77.0

In [9]:
fires.head(3)

,fire_name,fire_year,state,discovery_date,contain_date,acres_burned,fire_month,days_burning
0,Power,2004,CA,2004-10-06,2004-10-21,16823.0,10,15.0
1,Freds,2004,CA,2004-10-13,2004-10-17,7700.0,10,4.0
2,Bachelor,2004,NM,2004-07-20,2004-07-20,10.0,7,0.0


In [10]:
yearly_group = fires.groupby('fire_year')
yearly_sums = yearly_group.sum()
yearly_sums.head(3)

,acres_burned,fire_month,days_burning
fire_year,,,
1992,2123889.91,45643,6230.0
1993,2118394.10,52880,7283.0
1994,4033880.06,57669,20158.0


In [11]:
yearly_group = fires.groupby('fire_year', as_index=False)
yearly_sums = yearly_group.sum()
yearly_sums.head(3)

,fire_year,acres_burned,fire_month,days_burning
0,1992,2123889.91,45643,6230.0
1,1993,2118394.10,52880,7283.0
2,1994,4033880.06,57669,20158.0


### Pandas agg() method: Apply multiple aggregate methods

* **agg()** -> Applies an aggregate method or list of methods to a Series or DataFrame object.

In [12]:
monthly_group = fires.groupby(['state','fire_year','fire_month'])

In [13]:
monthly_group.agg(['sum','count','mean']).head(3)

C:\Users\Vijay Koju\AppData\Local\Temp\ipykernel_20340\3266516379.py:1: FutureWarning: ['fire_name', 'discovery_date', 'contain_date'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  monthly_group.agg(['sum','count','mean']).head(3)


acres_burned                    days_burning        \
                                    sum count         mean          sum count   
state fire_year fire_month                                                      
AK    1992      5                4202.0    15   280.133333        135.0    14   
                6               86401.0    26  3323.115385        417.0    25   
                7               48516.7    26  1866.026923        500.0    22   

                                       
                                 mean  
state fire_year fire_month             
AK    1992      5            9.642857  
                6           16.680000  
                7           22.727273

In [14]:
monthly_group.days_burning.agg(['sum','count','mean']).head(3)

sum  count       mean
state fire_year fire_month                         
AK    1992      5           135.0     14   9.642857
                6           417.0     25  16.680000
                7           500.0     22  22.727273

In [15]:
df = monthly_group.agg({'acres_burned':['sum','max','min'],
                   'days_burning':['sum','mean'],
                   'fire_name':'count'})
df.head(3)

acres_burned                days_burning  \
                                    sum      max   min          sum   
state fire_year fire_month                                            
AK    1992      5                4202.0   1410.0  10.0        135.0   
                6               86401.0  48087.0  10.0        417.0   
                7               48516.7  35090.0  10.0        500.0   

                                      fire_name  
                                 mean     count  
state fire_year fire_month                       
AK    1992      5            9.642857        14  
                6           16.680000        23  
                7           22.727273        26

### Pandas pivot() method

* **pivot(params)**  -> Pivots the data based on the index, columns, and values parameters.

Parameters:

* **index** -> The column or list of columns to use as the row index (no duplicates).
* **columns** -> The column or list of columns to use as the column index.
* **values** -> The column or list of columns to use to populate the new DataFrame. By default, all remaining columns are used.


In [16]:
states = ['AK','CA','ID','TX']
top_states = fires.groupby(['state','fire_year'], as_index=False).sum()
top_states = top_states.loc[top_states['state'].isin(states)]
#top_states = top_states.query('state in @states')
top_states.head(2)

,state,fire_year,acres_burned,fire_month,days_burning
0,AK,1992,142444.7,454,1145.0
1,AK,1993,686630.5,961,3373.0


In [17]:
top_states.pivot(index='fire_year', columns='state', values='acres_burned').head(2)

state,AK,CA,ID,TX
fire_year,,,,
1992,142444.7,289254.9,683495.2,31500.3
1993,686630.5,315011.1,7658.5,114265.5


### Pandas pivot_table() method

* **pivot_table(params)**  -> Produces a pivot table with an applied aggregate method.


Parameters:

* **index** -> The column or list of columns to use as the row index (allows duplicates).
* **columns** -> The column or list of columns to use as the column index.
* **values** -> The column or list of columns that contain the values to be aggregated. By default, all non-nuisance columns are aggregated.
* **aggfunc** -> The aggregate method or list of methods to be applied to each column in the values parameter
* **fill_value** -> The value to replace any missing values with in the resulting pivot table.

In [18]:
states = ['AK','CA','ID','TX']
fires_top_4 = fires.loc[fires['state'].isin(states)]
#fires_top_4 = fires.query['state' in @states]
fires_top_4 = fires_top_4.pivot_table(index='fire_year', columns='state', 
                                      values='acres_burned', aggfunc='sum')
fires_top_4.head(2)

state,AK,CA,ID,TX
fire_year,,,,
1992,142444.7,289254.9,683495.2,31500.3
1993,686630.5,315011.1,7658.5,114265.5


### Pandas cut() method: Create bins of equal size

* **cut(params)**  -> Bins the data into equal-sized bins.


Parameters:

* **x** -> The column that contains the data to be binned.
* **bins** -> The number of bins to create, or a list of values for the bin edges.
* **labels** -> The labels to use for the bins.
* **right** -> If set to False, the right edges are not included in the bins.

In [19]:
fires_filtered = fires.loc[(fires['fire_year']==2010) & (fires['days_burning'] > 0)].dropna()
#fires_filtered = fires.query('fire_year == 2010 and days_burning > 0').dropna()

In [20]:
pd.cut(fires_filtered.acres_burned, bins=4)

173229    (-296.103, 76535.75]
173230    (-296.103, 76535.75]
173231    (-296.103, 76535.75]
173232    (-296.103, 76535.75]
173233    (-296.103, 76535.75]
                  ...         
273969    (-296.103, 76535.75]
274046    (-296.103, 76535.75]
274049    (-296.103, 76535.75]
274081    (-296.103, 76535.75]
274116    (-296.103, 76535.75]
Name: acres_burned, Length: 1858, dtype: category
Categories (4, interval[float64, right]): [(-296.103, 76535.75] < (76535.75, 153061.5] < (153061.5, 229587.25] < (229587.25, 306113.0]]

In [21]:
pd.cut(fires_filtered.acres_burned, bins=[0,100000,200000,300000,400000])

173229    (0, 100000]
173230    (0, 100000]
173231    (0, 100000]
173232    (0, 100000]
173233    (0, 100000]
             ...     
273969    (0, 100000]
274046    (0, 100000]
274049    (0, 100000]
274081    (0, 100000]
274116    (0, 100000]
Name: acres_burned, Length: 1858, dtype: category
Categories (4, interval[int64, right]): [(0, 100000] < (100000, 200000] < (200000, 300000] < (300000, 400000]]

In [22]:
pd.cut(fires_filtered.acres_burned, bins=[0,100000,200000,300000,400000],
       labels=['small','medium','large','very large'])

173229    small
173230    small
173231    small
173232    small
173233    small
          ...  
273969    small
274046    small
274049    small
274081    small
274116    small
Name: acres_burned, Length: 1858, dtype: category
Categories (4, object): ['small' < 'medium' < 'large' < 'very large']

In [23]:
pd.cut(fires_filtered.acres_burned, bins=[0,100000,200000,300000,400000],
       labels=['small','medium','large','very large']).value_counts()

small         1855
medium           2
very large       1
large            0
Name: acres_burned, dtype: int64

## Pandas qcut() method: Create bins with equal numbers of unique items

* **qcut(params)**  -> Bins the data into quantiles with the same number of unique values in each bin. The number of rows in each bin will be skewed if there are duplicates.


Parameters:

* **x** -> The column that contains the data to be binned.
* **q** -> The number of quantiles to create.
* **labels** -> The labels to use for the bins.
* **duplicates** -> What to do with bins that have the same edges. The default is raise, which raises a ValueError. If set to drop, the non-unique bins are dropped.

In [24]:
pd.qcut(fires_filtered.acres_burned, q=4,
        labels=['small','medium','large','very large'])

173229         small
173230    very large
173231    very large
173232         large
173233        medium
             ...    
273969         small
274046        medium
274049        medium
274081         small
274116        medium
Name: acres_burned, Length: 1858, dtype: category
Categories (4, object): ['small' < 'medium' < 'large' < 'very large']

In [25]:
pd.qcut(fires_filtered.acres_burned, q=4,
        labels=['small','medium','large','very large']).value_counts()

small         466
medium        465
very large    464
large         463
Name: acres_burned, dtype: int64

In [26]:
fires_filtered['fire_size'] = pd.qcut(fires_filtered.acres_burned, q=4,
    labels=['small','medium','large','very large'])

In [27]:
pd.qcut(fires_filtered.days_burning, q=4,
        labels=['short','medium','long'], duplicates='drop').value_counts()

short     1018
long       433
medium     407
Name: days_burning, dtype: int64

## Pandas nlargest() method: Get the top n rows

* **nlargest(params)**  -> Returns the first n rows with the largest values in the specified columns.

Parameters:

* **n** -> The number of rows to return.
* **columns** -> The columns that determine which rows to keep.
* **keep** -> The rows to keep in the event of a tie. Possible values: first, last, and all. The default is first.

In [28]:
cars.nlargest(n=6, columns='enginesize')

,aspiration,carbody,enginesize,curbweight,price
49,std,sedan,326,3950,36000.0
73,std,sedan,308,3900,40960.0
74,std,hardtop,304,3715,45400.0
47,std,sedan,258,4066,32250.0
48,std,sedan,258,4066,35550.0
71,std,sedan,234,3740,34184.0


In [29]:
cars.nlargest(n=6, columns=['enginesize','price'])

,aspiration,carbody,enginesize,curbweight,price
49,std,sedan,326,3950,36000.0
73,std,sedan,308,3900,40960.0
74,std,hardtop,304,3715,45400.0
48,std,sedan,258,4066,35550.0
47,std,sedan,258,4066,32250.0
72,std,convertible,234,3685,35056.0


## Pandas pct_change() method: Calculate percent change

* **pct_change()**  -> Calculates the percent change from the previous row to the current row for a DataFrame or Series object.

In [30]:
df = fires[['state','fire_year','acres_burned']] \
    .groupby(['state','fire_year']).sum()
df.head()

acres_burned
state fire_year              
AK    1992           142444.7
      1993           686630.5
      1994           261604.7
      1995            43762.6
      1996           598407.2

In [31]:
df.pct_change()

acres_burned
state fire_year              
AK    1992                NaN
      1993           3.820330
      1994          -0.619002
      1995          -0.832715
      1996          12.673941
...                       ...
WY    2011           0.552941
      2012           2.582104
      2013          -0.888021
      2014          -0.866764
      2015           4.046849

[1148 rows x 1 columns]

## Pandas rank() method: Rank rows

* **rank(params)**  -> Computes numerical data ranks (1 through n) along an axis.

Parameters:

* **ascending** -> If False, ranks in descending order. If True (the default), ranks in ascending order.
* **method** -> How to rank the group of records that have ties. Possible values include average (the default), min, max, first, and dense.
* **pct** -> If True, displays the rankings in percentile form. False is the default.

In [32]:
df = fires.groupby('state').sum() \
    [['acres_burned','fire_year','days_burning']]
df.head(3)

,acres_burned,fire_year,days_burning
state,,,
AK,3.222601e+07,5683445,80268.0
AL,8.101628e+05,38336332,2886.0
AR,4.502221e+05,17960388,1132.0


In [33]:
df['acres_rank'] = df.acres_burned.rank(ascending=False)
df.head(3)

,acres_burned,fire_year,days_burning,acres_rank
state,,,,
AK,3.222601e+07,5683445,80268.0,1.0
AL,8.101628e+05,38336332,2886.0,23.0
AR,4.502221e+05,17960388,1132.0,27.0


In [34]:
df['days_rank'] = df.days_burning.rank(method='max')
df.sort_values('days_burning').head(4)

,acres_burned,fire_year,days_burning,acres_rank,days_rank
state,,,,,
RI,147.45,22092,0.0,51.0,1.0
VT,985.70,46240,6.0,50.0,3.0
CT,7358.20,364159,6.0,46.0,3.0
NH,1232.23,82240,9.0,49.0,4.0
